In [234]:
import numpy as np
import pandas as pd


In [235]:
import csv
import numpy as np

cnt = 1
csv_file_path = 'crop_production.csv'
states = {}
districts = {}
season_to_number = {}
crops = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in  csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        states[row[0].strip()] = 1
        districts[row[1].strip()] = 1
        season_to_number[row[3].strip()] = 1
        crops[row[4].strip()] = 1
        
cnt = 1
for val in states.keys():
    states[val] = cnt
    cnt = cnt+1
cnt = 1
for val in districts.keys():
    districts[val] = cnt
    cnt = cnt+1
cnt = 1

for val in season_to_number.keys():
    season_to_number[val] = cnt
    cnt = cnt+1
cnt = 1
for val in crops.keys():
    crops[val] = cnt
    cnt = cnt+1




In [236]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split


csv_file_path = 'crop_production.csv'

y = []
data = []
cnt = 0
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        try:
            y.append(float(row[6]))
        except:
            continue
        temp_row = row
        if len(temp_row[1]) ==0 : continue
        if len(temp_row[2]) ==0 : continue
        if len(temp_row[3]) ==0 : continue
        if len(temp_row[4]) ==0 : continue
        if len(temp_row[5]) ==0 : continue
        temp_row[1] = districts[temp_row[1].strip()]
        temp_row[2] = int(temp_row[2])
        temp_row[3] = season_to_number[temp_row[3].strip()]
        temp_row[4] = crops[temp_row[4].strip()]
        temp_row[5] = float(temp_row[5])
        
        temp1 = row[1:2]
        temp2 = row[3:6]
        data.append(temp1+temp2)
       
y = np.array(y)

data = np.array(data)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(data)



X_train, X_test, y_train, y_test = train_test_split(X_train_normalized, y, test_size = .25, random_state = 18)


In [237]:
# n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 

In [238]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    "n_estimators" : [100],
    'subsample' : [0.5,0.6,0.7],
    'colsample_bytree': [1.0],
}

model = XGBRegressor()
mse_grid = GridSearchCV(estimator=model, param_grid=param_grid_gb, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

mse_grid.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20)

print("Best parameters:", mse_grid.best_params_)


# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 5 folds for each of 3 candidates, totalling 15 fits
[0]	validation_0-rmse:14595496.50287
[1]	validation_0-rmse:11928986.65675
[2]	validation_0-rmse:10056462.91726


c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:8559693.45156
[4]	validation_0-rmse:7853138.11548
[5]	validation_0-rmse:7158915.79496
[6]	validation_0-rmse:6701383.99381
[7]	validation_0-rmse:6424676.87376
[8]	validation_0-rmse:6295446.26429
[9]	validation_0-rmse:6108406.13501
[10]	validation_0-rmse:6073179.51076
[11]	validation_0-rmse:6008379.77533
[12]	validation_0-rmse:6006967.14141
[13]	validation_0-rmse:5991683.80992
[14]	validation_0-rmse:5932640.04764
[15]	validation_0-rmse:5822952.72517
[16]	validation_0-rmse:5770079.27480
[17]	validation_0-rmse:5758446.85982
[18]	validation_0-rmse:5759260.10718
[19]	validation_0-rmse:5743296.78250
[20]	validation_0-rmse:5735738.59379
[21]	validation_0-rmse:5728982.27371
[22]	validation_0-rmse:5737934.41550
[23]	validation_0-rmse:5745984.56573
[24]	validation_0-rmse:5743279.38900
[25]	validation_0-rmse:5772531.23847
[26]	validation_0-rmse:5769317.58515
[27]	validation_0-rmse:5766855.90834
[28]	validation_0-rmse:5751968.87581
[29]	validation_0-rmse:5753212.31949
[30]	val

In [239]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

# param_random_gb = {
#               'learning_rate': np.arange(0.05,0.15, 0.05),
#               # 'n_estimators' : np.arange(200,1000,100)
#             #   'subsample' : np.arange(0.9,1, 0.05),
#             #   "colsample_bytree" : np.arange(0.9,1,0.05)
#               }

# # Regressor Instantiation
# gb = XGBRegressor()

# # Here we specified a total of 25 iterations
# mse_random = RandomizedSearchCV(estimator = gb, param_distributions= param_random_gb, n_iter = 25, 
#                                 scoring = 'neg_mean_squared_error', cv = 4, verbose = 1)

# mse_random.fit(X_train_normalized1,y_train1)

# print("Best parameter: ", mse_random.best_params_)

In [244]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


model = XGBRegressor(n_estimators=100 , colsample_bytree = 1 , subsample = 0.7)
# eval_set = [(X_test,y_test)]
model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=20,)

# print("Best parameter: ", mse_grid.best_params_)

[0]	validation_0-rmse:12545958.37380	validation_1-rmse:14595496.50287
[1]	validation_0-rmse:10108815.95915	validation_1-rmse:11928986.65675
[2]	validation_0-rmse:8567006.67013	validation_1-rmse:10056462.91726


c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:7361227.64364	validation_1-rmse:8559693.45156
[4]	validation_0-rmse:6641425.81406	validation_1-rmse:7853138.11548
[5]	validation_0-rmse:6108322.68219	validation_1-rmse:7158915.79496
[6]	validation_0-rmse:5733341.22151	validation_1-rmse:6701383.99381
[7]	validation_0-rmse:5499609.68319	validation_1-rmse:6424676.87376
[8]	validation_0-rmse:5282817.84778	validation_1-rmse:6295446.26429
[9]	validation_0-rmse:4931250.97952	validation_1-rmse:6108406.13501
[10]	validation_0-rmse:4769162.13824	validation_1-rmse:6073179.51076
[11]	validation_0-rmse:4747746.83175	validation_1-rmse:6008379.77533
[12]	validation_0-rmse:4537301.46713	validation_1-rmse:6006967.14141
[13]	validation_0-rmse:4365331.67905	validation_1-rmse:5991683.80992
[14]	validation_0-rmse:4296402.14093	validation_1-rmse:5932640.04764
[15]	validation_0-rmse:4153866.34226	validation_1-rmse:5822952.72517
[16]	validation_0-rmse:4091706.67966	validation_1-rmse:5770079.27480
[17]	validation_0-rmse:3910667.73315	vali

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [249]:
# training_error = 0
from sklearn.metrics import mean_squared_error
print("Training error:")
print(mean_squared_error(y_train,model.predict(X_train)))
print("Test error:")
print(mean_squared_error(y_test,model.predict(X_test)))
# Previously
# Training error:
# 2041076666.4211206
# Test error:
# 71631908865818.11
# Overfit
# Now
# Training error:
# 444956956552237.3
# Test error:
# 61499447426576.72
# Ok

Training error:
31279183727976.094
Test error:
20779240876814.168


In [253]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number['Whole Year']
Crop = crops['Cashewnut']
Area = 719.00
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_predict = np.zeros((1,4))
X_predict[0] = np.array([District_Name,Season,Crop,Area])
X_pred_normalized = scaler.transform(X_predict)

Production = model.predict(X_pred_normalized)
print(X_pred_normalized)
print(Production)

[[-1.65923103 -0.97610996 -1.21405365 -0.23761399]]
[148.93437]


In [247]:
import pickle
filename = "finalized_model.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)